In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai 

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [3]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3")
agent_executor = create_sql_agent(llm=llm, db=db, verbose=True)

In [4]:
agent_executor.invoke(
    "List the total sales per country. Which country's customers spent the most?"
)



> Entering new SQL Agent Executor chain...
Let's start by listing the tables in the database.

Thought: I will use sql_db_list_tables first!

Action: sql_db_list_tables
Action Input: (empty string)Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I have the list of tables, let me think about what to do next. Since I want to know the total sales per country and which country's customers spent the most, I should look at the tables related to orders and customers.

Action: sql_db_schema
Action Input: Invoice, Customer
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY 

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': "The customers from USA spent the most, with a total of $523.06. However, since there is no data about USA, it seems like there's an error in the database or the query."}

In [5]:
agent_executor.invoke("Describe the playlisttrack table")



> Entering new SQL Agent Executor chain...
Let's start by getting a list of all the tables in the database.

Thought: I should get a list of all the tables first, then use that list to decide which table(s) to query for their schema.

Action: sql_db_list_tables
Action Input: empty stringAlbum, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackLet's use the list of tables to decide which table(s) to query for their schema.

Action: sql_db_schema
Action Input: Playlist, PlaylistTrack
CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/


CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows 

{'input': 'Describe the playlisttrack table',
 'output': 'The `PlaylistTrack` table is a junction table that links playlists to tracks. It has two columns: `PlaylistId` and `TrackId`, which are both foreign keys referencing the `Playlist` and `Track` tables respectively. The primary key of this table is the combination of these two foreign keys.'}

In [6]:
agent_executor.invoke("List all artists.")



> Entering new SQL Agent Executor chain...
Let's start by listing the tables in the database.

Action: sql_db_list_tables
Action Input: empty stringAlbum, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I have a list of all the tables, I should look at the schema to see what columns and data types are available. This will help me figure out how to query for artists.

Action: sql_db_schema
Action Input: Artist, Album
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	A

{'input': 'List all artists.',
 'output': 'AC/DC, Accept, Aerosmith, Alanis Morissette, Alice In Chains, Antônio Carlos Jobim, Apocalyptica, Audioslave, BackBeat, Billy Cobham'}

In [7]:
agent_executor.invoke("How many albums does alis in chain have?")



> Entering new SQL Agent Executor chain...
Let's start by getting a list of tables in the database.

Thought: I'll use sql_db_list_tables to get a list of tables.
Action: sql_db_list_tables
Action Input: (empty string)Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I have the list of tables, I need to think about which table(s) might be relevant for this question. The question is asking about albums by a specific artist (alis in chain), so I'll focus on tables related to artists and albums.

Action: sql_db_schema
Action Input: Artist, Album

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"A

{'input': 'How many albums does alis in chain have?',
 'output': 'alis in chain (ArtistId 3) has at least one album titled "Big Ones". Since the query only returned one result, I\'m not sure if there are more than one album by this artist. \n\nPlease let me know what to do next!'}

In [8]:
agent_executor.invoke("Find all albums for the artist 'AC/DC'.")



> Entering new SQL Agent Executor chain...
Let's start by listing the tables in the database using `sql_db_list_tables`.

Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I have a list of tables, I should check the schema for the relevant tables. Since we're looking for albums by AC/DC, I think `Album` and `Track` are the most relevant tables.

Action: sql_db_schema
Action Input: Album, Track
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NUL

{'input': "Find all albums for the artist 'AC/DC'.",
 'output': "For Those About to Rock We Salute You and Let There Be Rock are the two albums by AC/DC.\n\nNote: I limited my query to only return the ArtistId and Title columns from the Album table, as that's all we need to answer this question."}

In [9]:
agent_executor.invoke("List all tracks in the 'Rock' genre.")



> Entering new SQL Agent Executor chain...
Let's start by getting a list of available tables in the database.

Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I have a list of tables, I should look at the schema of the relevant tables to see what columns they contain. In this case, I'm interested in finding all tracks in the 'Rock' genre, so I should focus on the Track and Genre tables.

Action: sql_db_schema
Action Input: Track, Genre
CREATE TABLE "Genre" (
	"GenreId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("GenreId")
)

/*
3 rows from Genre table:
GenreId	Name
1	Rock
2	Jazz
3	Metal
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10

{'input': "List all tracks in the 'Rock' genre.",
 'output': "The tracks in the 'Rock' genre are:\n\nFor Those About To Rock (We Salute You)\nBalls to the Wall\nFast As a Shark\nRestless and Wild\nPrincess of the Dawn\nPut The Finger On You\nLet's Get It Up\nInject The Venom\nSnowballed\nEvil Walks\n\nThese are the tracks in the 'Rock' genre, which is what you asked for."}

In [10]:
agent_executor.invoke("Find the total duration of all tracks.")



> Entering new SQL Agent Executor chain...
Let's get started.

Thought: Since we're looking for the total duration of all tracks, it seems that we should focus on a table related to music or audio tracks. Let's check the list of tables in the database first.

Action: sql_db_list_tables
Action Input: (empty string)Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I know the tables available in the database, I should look at the schema of the relevant tables to see what columns are available. Let's check the schema of the tables related to music or audio tracks.

Action: sql_db_schema
Action Input: Track
CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FORE

{'input': 'Find the total duration of all tracks.', 'output': '1378778040'}

In [12]:
agent_executor.invoke("How many tracks are there in the album with ID 5?")



> Entering new SQL Agent Executor chain...
Let's get started.

Thought: Since the question is asking about an album with ID 5, I think I'll need to check which tables are available in the database and then look at the schema of those tables to see if they have information about albums and tracks.

Action: sql_db_list_tables
Action Input: empty stringAlbum, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackI've got a list of tables in the database. Now I need to figure out which tables have information about albums and tracks.

Action: sql_db_schema
Action Input: Album, Track, PlaylistTrack
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREAT

{'input': 'How many tracks are there in the album with ID 5?',
 'output': '15\n\nI know the final answer because I constructed a query using the information from the schema of the relevant tables. The query was double-checked with sql_db_query_checker and then executed with sql_db_query.'}

In [13]:
agent_executor.invoke("Find the total number of invoices.")



> Entering new SQL Agent Executor chain...
Let's get started.

Thought: Since we're looking for the total number of invoices, I think the relevant table is likely "invoices". Before querying anything, let me make sure I have a list of all tables in the database. I'll use `sql_db_list_tables` to do that.

Action: sql_db_list_tables
Action Input: ""Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I have a list of all tables in the database, it seems like "invoices" is indeed a relevant table. Let me check its schema to see what columns are available.

Action: sql_db_schema
Action Input: invoicesError: table_names {'invoices'} not found in databaseIt looks like there's an issue with the `sql_db_schema` action. The error message indicates that the "invoices" table was not found in the database. This is likely because `sql_db_list_tables` did not return this table, which means it may not exist in the database after 

{'input': 'Find the total number of invoices.', 'output': '412'}

In [15]:
agent_executor.invoke("Who are the top 5 customers by total purchase?")



> Entering new SQL Agent Executor chain...
Let's start by getting a list of all the tables in the database using `sql_db_list_tables`.

Question: Who are the top 5 customers by total purchase?
Thought: I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I have a list of all the tables in the database, I should look at the schema of the tables related to customers and purchases. This will help me determine which columns to query.

Action: sql_db_schema
Action Input: Customer, Invoice
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode

{'input': 'Who are the top 5 customers by total purchase?',
 'output': "The top 5 customers by total purchase are Helena Holý, Richard Cunningham, Luis Rojas, Ladislav Kovács, and Hugh O'Reilly."}

In [16]:
agent_executor.invoke("Which albums are from the year 2000?")



> Entering new SQL Agent Executor chain...
Let's get started.

Thought: Before I start querying, I need to know which tables exist in the database and which ones are relevant to this question. 

Action: sql_db_list_tables
Action Input: empty stringAlbum, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I know the tables in the database, I need to identify which ones are relevant to this question. Since we're looking for albums from a specific year, the "Album" and possibly "Track" tables might be useful.

Action: sql_db_schema
Action Input: Album, Track
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Track" (


{'input': 'Which albums are from the year 2000?',
 'output': "I don't know\n\nAs the query executed did not return any album from the year 2000."}

In [17]:
agent_executor.invoke("How many employees are there")



> Entering new SQL Agent Executor chain...
Let's start by listing all the tables in the database using `sql_db_list_tables`.

Thought: Next, I'll check if a table called "employees" exists. If it does, I'll query its schema to see what columns are available.

Action: sql_db_list_tables
Action Input: (empty string)Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I have the list of tables, I can check if "employees" is one of them. Let me query its schema to see what columns are available.

Action: sql_db_schema
Action Input: Employee
CREATE TABLE "Employee" (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"ReportsTo" INTEGER, 
	"BirthDate" DATETIME, 
	"HireDate" DATETIME, 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" 

{'input': 'How many employees are there', 'output': '8'}